L'objectif de ce notebook est de classifier les images ingredients et valeurs nutritionnelles en utilisant OCR avec la bibliothèque pytesseract. Les résultats sont peu pertinents.

# Imports et initialisation

In [ ]:
import os
from google.colab import drive
drive.mount('drive')

!mkdir -p drive -v

cwd = os.getcwd()

monchemin = 'drive/My Drive/Centrale Marseille/3A/Projet 3A/dataset'

dir_path=os.path.join(cwd, monchemin)
dirs =os.listdir(dir_path)
print(dir_path)
print (dirs)

os.chdir(dir_path)

!export PYTHONPATH=$PYTHONPATH:dir_path

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
/content/drive/My Drive/Centrale Marseille/3A/Projet 3A/dataset
['front', 'ingredients', 'nutrition', 'models', 'tessdata-main', 'eng.traineddata', 'fra.traineddata']


In [ ]:
! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install Pillow
! pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2build2).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from PIL import Image
import requests
from io import BytesIO
import pytesseract
import numpy as np
import re
from os import listdir
from os.path import isfile, join

In [ ]:
#!wget https://github.com/tesseract-ocr/tessdata/archive/master.zip
#!unzip master.zip
!cp tessdata-main/fra.traineddata /usr/share/tesseract-ocr/4.00/tessdata/
!cp tessdata-main/eng.traineddata /usr/share/tesseract-ocr/4.00/tessdata/
!cp tessdata-main/spa.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

# Reconnaissance de caractères

## Depuis la base de données

In [ ]:
img = Image.open('ingredients/ing_197.jpg') #Image retournée

In [ ]:
text = pytesseract.image_to_string(img.transpose(Image.Transpose.ROTATE_180))
text

"WO SUES 2001 S SIAQID Bf0s ap saureiB,\n\nape: WeypDe ‘A1 9P 2 ugioid s\nmmnwm‘usad‘nxs'f\nsaquay Sau) 12 e VODILR (S\nxm,ﬁﬁapwm)% m»\n\n   \n\x0c"

In [ ]:
text = text.replace('\n', ' ')
text

'a selection of milk, dark and white chocolate coated biscuits 27 biscuts - 12 varieties INGREDIENTS Milk Chocolate (32%) (Sugar - Cocoa Butter - Dried Whole Milk - Cocoa Mass - Dried Whey Dried Skimmed Milk - Emuisifier: Soya Lecithin) - ‘Chocolate (17%) (Sugar - Cocoa Mass - Cocoa. Butter - Butter Ol (Milk) - Emuisifier: Soya Lecithin - Vanila Flavouring) - Wheatflour contains Gluten (with Wheatflour, Cacium Garbonate, Iron, Niacin, Thiamin) - White Chocolate (13%) (Sugar Cocoa Butter - Dried Skimmed Milk - Butter Oil (Milk) - Dried Whole Milk - Emulsifier: Soya Lecithin - Vanilla Flavouring) - Sugar - Paim Oil- Butter (Milk) - Paim Kernel Oil - Paim Fat - Gocoa Powder Partially Inverted Sugar Syrup - Glucose Syrup - Cornflour - Dried ‘Skimmed Milk - Dried Whole Milk - Oatmeal (contains Gluten)  Salt - Ginger - Raising Agent: Sodium Bicarbonate, E450, E503 Dextrose - Cocoa Mass - Emulsifier: Soya Lecithin - Ground Ginger -  Pasteurised Free Range Egg - Flavourings - Orange Peel - Mol

In [ ]:
def characters_recognition(img):
  rotated_images = [img, img.transpose(Image.Transpose.ROTATE_90), img.transpose(Image.Transpose.ROTATE_180), img.transpose(Image.Transpose.ROTATE_270)]
  associated_texts = []
  len_texts = []

  for i in range(len(rotated_images)):
    rotated_images[i]
    text = pytesseract.image_to_string(rotated_images[i])
    associated_texts.append(text)
    len_texts.append(len(text))

  return associated_texts[np.argmax(len_texts)]

In [ ]:
characters_recognition(img)

'a selection of milk, dark and white chocolate coated biscuits\n27 biscuts - 12 varieties\nINGREDIENTS Milk Chocolate (32%) (Sugar - Cocoa\nButter - Dried Whole Milk - Cocoa Mass - Dried Whey\nDried Skimmed Milk - Emuisifier: Soya Lecithin) -\n‘Chocolate (17%) (Sugar - Cocoa Mass - Cocoa.\nButter - Butter Ol (Milk) - Emuisifier: Soya Lecithin - Vanila\nFlavouring) - Wheatflour contains Gluten (with Wheatflour, Cacium\nGarbonate, Iron, Niacin, Thiamin) - White Chocolate (13%) (Sugar\nCocoa Butter - Dried Skimmed Milk - Butter Oil (Milk) - Dried Whole\nMilk - Emulsifier: Soya Lecithin - Vanilla Flavouring) - Sugar - Paim\nOil- Butter (Milk) - Paim Kernel Oil - Paim Fat - Gocoa Powder\nPartially Inverted Sugar Syrup - Glucose Syrup - Cornflour - Dried\n‘Skimmed Milk - Dried Whole Milk - Oatmeal (contains Gluten)\n\nSalt - Ginger - Raising Agent: Sodium Bicarbonate, E450, E503\nDextrose - Cocoa Mass - Emulsifier: Soya Lecithin - Ground Ginger -\n\nPasteurised Free Range Egg - Flavourings -

# Baseline

In [ ]:
def baseline_1(img):
  associated_text = characters_recognition(img)
  return associated_text, ("ingredient" in associated_text.lower()) or ("ingrédient" in associated_text.lower())

In [ ]:
baseline_1(img)[1]

True

In [ ]:
fichiers = [f for f in listdir("ingredients") if isfile(join("ingredients", f))]

In [ ]:
errors = 0
taille = len(fichiers)

for i in range(taille):
  print(str(i) + '/' + str(taille))
  text = baseline_1(Image.open('ingredients/' + fichiers[i]))
  if text[1] == False:
    errors += 1
    print(fichiers[i] + ' --------------------------------------------------------------------------------> erreur')
    print(text[0])

0/440
1/440
ing_17.jpg --------------------------------------------------------------------------------> erreur
  

  

Sucre. farine de BLE. eau. purée de passion 14% (jus
< e frat Ge la passion 80%. sucre) OEUR

S margaring (nutes st graisses végatales raffinées

P iractionnees (pame. colza). eau. sel smulsfiants

mono et ciglvoerides dacides gras, Sarrecteut

Jaciaité - acice Gitrique. colorant : béta-carotene)

SEURRE, sucre giece (fécule de pomes datere),

 
 

   
 

  

  

 

—

 

2/440
ing_20.jpg --------------------------------------------------------------------------------> erreur
  

Framborses 45% farine de FROMENT (agent de
tratement de [a farine : E300). Sucte, si1op &6
glucose-ructoss, eau, BEURRE, hule de paime,
Graisse de paime fractionnée, hulle de o EUF
S nier, poudre de pett LAT enichie en protéines, b
E440, aciifiant - E330, cortecleurs SRt
332 nt ; E333,LACTOSERAV!

 

  

  
       

  

3/440
ing_19.jpg ------------------------------------------------------

In [ ]:
print(errors)

292
